In [2]:
import glob
import re
import matplotlib as plt
import numpy as np
import xarray as xr
import pandas as pd

In [3]:
import dask 
from dask_jobqueue import PBSCluster
from dask.distributed import Client
from dask.distributed import performance_report

In [4]:
######## File paths ################
lustre_scratch    = "/lustre/desc1/scratch/harshah"
era5_surface_data = "/gdex/data/special_projects/harshah/ARCO/e5.oper.an.sfc"

In [16]:
#CONUS bounds
lat_top  = 50
lat_bottom = 20
lon_left  = (360-65)
lon_right = (360-130)
print(lon_right,lon_left)

230 295


In [6]:
cluster = PBSCluster(
        job_name = 'dask-osdf-25',
        cores = 1,
        memory = '4GiB',
        processes = 1,
        local_directory = lustre_scratch + '/dask/spill',
        log_directory = lustre_scratch + '/dask/logs/',
        resource_spec = 'select=1:ncpus=1:mem=4GB',
        queue = 'casper',
        walltime = '3:00:00',
        #interface = 'ib0'
        interface = 'ext'
    )

In [7]:
client = Client(cluster)
# Scale the cluster and display cluster dashboard URL
n_workers =5
cluster.scale(n_workers)
client.wait_for_workers(n_workers = n_workers)
cluster

PBSCluster(e5bfdb03, 'tcp://128.117.208.96:33219', workers=5, threads=5, memory=20.00 GiB)

## GMST functions

In [8]:
# GMST function ###
# calculate global means
def get_lat_name(ds):
    for lat_name in ['lat', 'latitude']:
        if lat_name in ds.coords:
            return lat_name
    raise RuntimeError("Couldn't find a latitude coordinate")

def global_mean(ds):
    lat = ds[get_lat_name(ds)]
    weight = np.cos(np.deg2rad(lat))
    weight /= weight.mean()
    #other_dims = set(ds.dims) - {'time'}
    other_dims = set(ds.dims)
    return (ds * weight).mean(other_dims)

## Load data and compute GMST

In [9]:
%%time
temp_2m = xr.open_zarr(era5_surface_data + '/e5.oper.an.sfc.2t.zarr')  
temp_2m = temp_2m.VAR_2T
temp_2m

CPU times: user 1.41 s, sys: 288 ms, total: 1.7 s
Wall time: 3.73 s


<xarray.DataArray 'VAR_2T' (time: 749472, latitude: 721, longitude: 1440)> Size: 3TB
dask.array<open_dataset-VAR_2T, shape=(749472, 721, 1440), dtype=float32, chunksize=(27, 480, 241), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 6MB 1940-01-01 ... 2025-06-30T23:00:00
Attributes: (12/14)
    QuantizeGranularBitGroomNumberOfSignificantDigits:  7
    ecmwf_local_table:                                  128
    ecmwf_parameter:                                    167
    grid_specification:                                 0.25 degree x 0.25 de...
    long_name:                                          2 metre temperature
    maximum_value:                                      320.42938232421875
    ...                                                 ...
    rda_dataset:                                        ds633.0
    rda_dataset_doi:                                    DOI: 10.5065/BH6N-5N20
    rda_dataset_group:                                  ERA5 atmospheric surf...
    rda_dataset_url:                                    https:/rda.ucar.edu/d...
    short_name:                                         2t
    units:                                              K

In [17]:
temp_2m_1950_conus = temp_2m.sel(time=temp_2m.time.dt.year==1950).sel(latitude=slice(lat_top,lat_bottom))\
                       .sel(longitude = slice(lon_right,lon_left))
temp_2m_1950_conus

<xarray.DataArray 'VAR_2T' (time: 8760, latitude: 121, longitude: 261)> Size: 1GB
dask.array<getitem, shape=(8760, 121, 261), dtype=float32, chunksize=(26, 121, 217), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 968B 50.0 49.75 49.5 49.25 ... 20.5 20.25 20.0
  * longitude  (longitude) float64 2kB 230.0 230.2 230.5 ... 294.5 294.8 295.0
  * time       (time) datetime64[ns] 70kB 1950-01-01 ... 1950-12-31T23:00:00
Attributes: (12/14)
    QuantizeGranularBitGroomNumberOfSignificantDigits:  7
    ecmwf_local_table:                                  128
    ecmwf_parameter:                                    167
    grid_specification:                                 0.25 degree x 0.25 de...
    long_name:                                          2 metre temperature
    maximum_value:                                      320.42938232421875
    ...                                                 ...
    rda_dataset:                                        ds633.0
    rda_dataset_doi:                                    DOI: 10.5065/BH6N-5N20
    rda_dataset_group:                                  ERA5 atmospheric surf...
    rda_dataset_url:                                    https:/rda.ucar.edu/d...
    short_name:                                         2t
    units:                                              K

In [19]:
%%time
# Grid cells are spatially weighted and each time step gets the same weight 
conus_mst_1950 = global_mean(temp_2m_1950_conus)
conus_mst_1950

CPU times: user 357 ms, sys: 28 ms, total: 385 ms
Wall time: 443 ms


<xarray.DataArray ()> Size: 8B
dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

- This is sizeable dask graph!
- Let us try to compute it!

In [21]:
%%timeit -n 2 -r 3 
conus_mst_copy = conus_mst_1950.copy()
conus_mst_copy.values

12 s ± 1.17 s per loop (mean ± std. dev. of 3 runs, 2 loops each)


In [25]:
conus_mst_1950.values

array(288.30948011)